# 실습 5: Tools 생성

## 목표
- Disaster Tools 구현 (소방서 검색, 기상 정보, 위키피디아)
- Browser Tool 구현 (뉴스 검색)
- Tool 동작 테스트

## 5-1. 설정 확인

In [ ]:
from workshop_config import REGION, WINDY_API_KEY

print(f"🌍 Region: {REGION}")
print(f"🌤️ Windy API Key: {'설정됨' if WINDY_API_KEY else '⚠️ 미설정'}")

---
## 5-2. Disaster Tools 생성

In [ ]:
%%writefile ../agent/deploy/runtime/tool_use/disaster_tools.py
"""화재 대응 도구 모음"""

from typing import Dict, Any
import wikipedia as wiki
import requests
from strands import tool
import boto3
import json
import math
from config import WINDY_API_KEY, WINDY_API_URL


@tool
def wikipedia_search(query: str) -> Dict[str, Any]:
    """위키피디아에서 정보를 검색합니다.
    Args:
        query: 검색어
    Returns:
        검색 결과를 담은 딕셔너리
    """
    try:
        wiki.set_lang("ko")
        page = wiki.page(query)
        summary = page.summary
        if len(summary) > 500:
            summary = summary[:500] + "..."
        return {
            "success": True,
            "title": page.title,
            "summary": summary,
            "url": page.url
        }
    except wiki.exceptions.DisambiguationError as e:
        return {
            "success": False,
            "error": "여러 결과가 발견되었습니다",
            "options": e.options[:5]
        }
    except Exception as e:
        return {"success": False, "error": str(e)}


@tool
def find_fire_station(address: str) -> str:
    """화재 발생 시 주소를 기반으로 가까운 소방서 5곳을 벡터 검색으로 찾습니다.
    Args:
        address: 한국 주소 (예: 서울특별시 서초구 방배중앙로 06681)
    Returns:
        소방서 5곳의 정보 문자열 (LLM이 최적의 소방서를 판단할 수 있도록)
    """
    try:
        print(f"🔍 주소 검색: {address}")
        bedrock = boto3.client("bedrock-runtime", region_name="us-west-2")
        s3vectors = boto3.client("s3vectors", region_name="us-west-2")
        
        # 주소를 임베딩으로 변환
        embedding_response = bedrock.invoke_model(
            modelId="amazon.titan-embed-text-v2:0",
            body=json.dumps({
                "inputText": address,
                "dimensions": 1024,
                "normalize": True
            })
        )
        embedding_body = json.loads(embedding_response["body"].read())
        query_embedding = embedding_body["embedding"]
        print(f"✅ 임베딩 생성 완료")
        
        # S3 Vectors로 유사도 검색
        search_response = s3vectors.query_vectors(
            vectorBucketName="firestation-location-xy",
            indexName="fire-station",
            queryVector={"float32": query_embedding},
            topK=5,
            returnMetadata=True
        )
        
        if search_response.get("vectors"):
            result = f"화재 발생 주소: {address}\n\n가까운 소방서 5곳:\n\n"
            for idx, vector_item시작  in enumerate(search_response["vectors"], 1):
                metadata = vector_item.get("metadata", {})
                score = vector_item.get("score", 0)
                
                name = metadata.get("name", "소방서")
                station_address = metadata.get("address", "")
                lat = metadata.get("latitude", "")
                lon = metadata.get("longitude", "")
                phone = metadata.get("phone", "")
                
                result += f"{idx}. {name}\n"
                result += f"   - 위도: {lat}, 경도: {lon}\n"
                result += f"   - 주소: {station_address}\n"
                result += f"   - 전화번호: {phone}\n\n"
                print(f"✅ 소방서 {idx}: {name} (점수: {score:.4f})")
            return result
        return "소방서를 찾을 수 없습니다."
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return f"오류 발생: {str(e)}"


@tool
def get_weather_info(latitude: float, longitude: float) -> str:
    """Windy API를 사용해 기상 정보를 조회합니다.
    Args:
        latitude: 위도 (prompt에서 받은 값)
        longitude: 경도 (prompt에서 받은 값)
    Returns:
        기상 정보 문자열
    """
    try:
        print(f"🌤️ 기상 정보 조회: 위도={latitude}, 경도={longitude}")
        payload = {
            "lat": latitude,
            "lon": longitude,
            "model": "gfs",
            "parameters": ["wind", "temp", "rh", "pressure"],
            "levels": ["surface"],
            "key": WINDY_API_KEY
        }
        response = requests.post(WINDY_API_URL, json=payload, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data and data.get("ts"):
            idx = 0
            wind_u = data.get("wind_u-surface", [0])[idx]
            wind_v = data.get("wind_v-surface", [0])[idx]
            temp_k = data.get("temp-surface", [273.15])[idx]
            humidity = data.get("rh-surface", [0])[idx]
            pressure = data.get("pressure-surface", [1013])[idx]

            wind_speed = math.sqrt(wind_u**2 + wind_v**2)
            wind_dir = (math.atan2(wind_u, wind_v) * 180 / math.pi + 180) % 360
            temp_c = temp_k - 273.15

            address = get_address_from_coords(latitude, longitude).replace(',', ' ')

            result = f"위도: {latitude}, 경도: {longitude}\n"
            result += f"주소: {address}\n"
            result += f"온도: {temp_c:.1f}°C\n"
            result += f"풍속: {wind_speed:.1f} m/s\n"
            result += f"풍향: {wind_dir:.0f}°\n"
            result += f"습도: {humidity}%\n"
            result += f"기압: {pressure} hPa"
            print(f"✅ 기상 정보 조회 완료")
            return result
        return "기상 정보를 가져올 수 없습니다."
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return f"기상 정보 조회 오류: {str(e)}"


def get_address_from_coords(lat: float, lon: float) -> str:
    """좌표를 주소로 변환합니다 (Nominatim 사용)"""
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&accept-language=ko"
        response = requests.get(url, headers={"User-Agent": "DisasterMonitoring/1.0"}, timeout=10)
        data = response.json()
        
        addr = data.get("address", {})
        parts = []
        
        # 시/도
        if addr.get("city"):
            parts.append(addr["city"])
        elif addr.get("province"):
            parts.append(addr["province"])
        
        # 구/군
        if addr.get("borough"):
            parts.append(addr["borough"])
        elif addr.get("suburb"):
            parts.append(addr["suburb"])
        
        # 도로명
        if addr.get("road"):
            parts.append(addr["road"])
        
        # 번지
        if addr.get("house_number"):
            parts.append(addr["house_number"])
        
        # 우편번호
        if addr.get("postcode"):
            parts.append(addr["postcode"])
        
        result = " ".join(parts) if parts else data.get("display_name", f"{lat:.6f}, {lon:.6f}")
        return result
        
    except Exception as e:
        print(f"❌ 주소 변환 오류: {str(e)}")
        return f"{lat:.6f}, {lon:.6f}"

---
## 5-3. Browser Tool 생성

In [ ]:
%%writefile ../agent/deploy/runtime/tool_use/browser_tool/playwright.py
"""브라우저 도구 에이전트"""

from strands import tool, Agent
from strands_tools.browser import AgentCoreBrowser
from config import BROWSER_MODEL_ID, BROWSER_SYSTEM_PROMPT


def create_agent():
    agent_core_browser = AgentCoreBrowser(region="us-west-2")
    agent = Agent(
        tools=[agent_core_browser.browser],
        model=BROWSER_MODEL_ID,
        system_prompt=BROWSER_SYSTEM_PROMPT
    )
    return agent


@tool
def browser_tool_agent(prompt: str):
    """소방 대응 지휘를 위한 지역 상황을 다음 웹사이트를 호출,방문해 정보를 수집합니다.
    visit web site: https://news.google.com/search?q={prompt}&hl=ko&gl=KR&ceid=KR%3Ako
    이때 q parameter는 prompt를 입력하는데 아래의 규칙을 따른다.
    prompt 처리:
    - "서울특별시 강남구 남부순환로 06284" → "서울특별시 강남구"
    - 번지와 도로명 제거, 시/구 단위로 검색
    
    Args:
        prompt: 검색할 지역과 수집할 정보 유형
    
    Returns:
        검색 결과 종합 정보 (기사 제목, 업데이트 시간, 기사 출처)
    """
    try:
        strands_agent = create_agent()
        print(f"✅ 브라우저 도구 실행: {prompt}")
        response = strands_agent(prompt)
        return response.message["content"][0]["text"] if response.message.get("content") else str(response)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return f"오류 발생: {str(e)}"

---
## 5-4. Tool 테스트

생성한 Tools가 정상 동작하는지 테스트합니다.

In [ ]:
import sys
sys.path.insert(0, '../agent/deploy/runtime')

from tool_use.disaster_tools import wikipedia_search, find_fire_station, get_weather_info
print("✅ Tools 로드 완료")

### 테스트 1: 위키피디아 검색

In [ ]:
print("🔍 위키피디아 검색 테스트: '화재'\n")
result = wikipedia_search("화재")
print(result)

### 테스트 2: 소방서 검색 (S3 Vectors)

In [ ]:
print("🚒 소방서 검색 테스트: '서울특별시 강남구 테헤란로'\n")
result = find_fire_station("서울특별시 강남구 테헤란로")
print(result)

### 테스트 3: 기상 정보 (Windy API)

In [ ]:
print("🌤️ 기상 정보 테스트: 서울 (37.5665, 126.9780)\n")
result = get_weather_info(37.5665, 126.9780)
print(result)

### 테스트 4: 브라우저 도구 테스트

In [ ]:
print("📰 Browser Tool 테스트")
from tool_use.browser_tool.playwright import browser_tool_agent
result = browser_tool_agent("한글로, Visit the website [https://news.google.com/home?hl=ko&gl=KR&ceid=KR:ko] and explain the news contents. If you take a screenshot, please inform me of the screenshot URL so I can verify it.")
print(result)

---
## ✅ 실습 5 완료!

### 생성된 파일
- `agent/deploy/runtime/tool_use/disaster_tools.py`
- `agent/deploy/runtime/tool_use/browser_tool/playwright.py`

👉 다음: **lab6_agent_runtime.ipynb**